In [18]:
import pandas
from terminusdb_client import WOQLQuery as WQ
from terminusdb_client import WOQLDataFrame as WDF
from terminusdb_client import WOQLClient, WOQLView

server_url = "https://127.0.0.1:6363"
user = "admin"
account = "admin"
key = "root"
dbid = "seshat_combined"
repository = "local"

client = WOQLClient(server_url)
client.connect(user=user,account=account,key=key,db=dbid)

client.checkout("conflict")

'conflict'

In [22]:
date_query = WQ().select("v:Start_Integer", "v:End_Integer").woql_and(
            WQ().triple("v:Polity", "type", "scm:Polity"),
            WQ().triple("v:Polity", "scm:start", "v:Start_Range"),
            WQ().cast("v:Start_Range", "xsd:string", "v:Start_String"),
            WQ().cast("v:Start_String", "xsd:integer", "v:Start_Integer"),
            WQ().greater(600, "v:Start_Integer"),
            WQ().triple("v:Polity", "scm:end", "v:End_Range"),
            WQ().cast("v:End_Range", "xsd:string", "v:End_String"),
            WQ().cast("v:End_String", "xsd:integer", "v:End_Integer"),
            WQ().greater("v:End_Integer", 400)
        )

In [23]:
polity_query = (WQ().woql_and(
        WQ().triple("v:Polity", "type", "scm:Polity"),
        WQ().opt().woql_and(
            WQ().triple("v:Polity", "successor", "v:Successor"),
            WQ().triple("v:Successor", "label", "v:Slab"),
        ),
        WQ().opt().woql_and(
            WQ().triple("v:Polity", "predecessor", "v:Predecessor"),
            WQ().triple("v:Predecessor", "label", "v:Plab"),
        ),
        WQ().triple("v:Polity", "label", "v:Lab"),
        WQ().opt().woql_and(
            WQ().triple("v:Polity", "professional_soldiers", "v:Variable"),
               WQ().triple("v:Variable", "epistemicState", "v:Presence")
        ),
        WQ().opt().woql_and(
          WQ().triple("v:Conflict", "scm:belligerent", "v:Polity"),
          WQ().triple("v:Conflict", "type", "scm:Battle"), 
          WQ().triple("v:Conflict", "label", "v:Blab")
        )
     )
)
result = polity_query.execute(client)

In [24]:
WDF.result_to_df(result)

,Blab,Conflict,Lab,Plab,Polity,Predecessor,Presence,Slab,Successor,Variable
0,system:unknown,system:unknown,Afsharid Dynasty,system:unknown,terminusdb:///data/afafshr,system:unknown,system:unknown,system:unknown,system:unknown,system:unknown
1,Battle of Amritsar,terminusdb:///data/Battle_of_Amritsar_(1757),Durrani Empire,Afsharid Dynasty,terminusdb:///data/afdurrn,terminusdb:///data/afafshr,terminusdb:///schema#present,system:unknown,system:unknown,terminusdb:///data/ee7f298d27e5c195e28a076c6ae...
2,Battle of Attock,terminusdb:///data/Battle_of_Attock,Durrani Empire,Afsharid Dynasty,terminusdb:///data/afdurrn,terminusdb:///data/afafshr,terminusdb:///schema#present,system:unknown,system:unknown,terminusdb:///data/ee7f298d27e5c195e28a076c6ae...
3,Battle of Attock,terminusdb:///data/Battle_of_Attock_(1758),Durrani Empire,Afsharid Dynasty,terminusdb:///data/afdurrn,terminusdb:///data/afafshr,terminusdb:///schema#present,system:unknown,system:unknown,terminusdb:///data/ee7f298d27e5c195e28a076c6ae...
4,Battle of Kup,terminusdb:///data/Battle_of_Kup,Durrani Empire,Afsharid Dynasty,terminusdb:///data/afdurrn,terminusdb:///data/afafshr,terminusdb:///schema#present,system:unknown,system:unknown,terminusdb:///data/ee7f298d27e5c195e28a076c6ae...
...,...,...,...,...,...,...,...,...,...,...
940,system:unknown,system:unknown,Kingdom of Saba and Dhu Raydan,Yemen - Mukarrib Period,terminusdb:///data/yesaray,terminusdb:///data/yemukar,system:unknown,system:unknown,system:unknown,system:unknown
941,system:unknown,system:unknown,Yemen - Tahirid Dynasty,Yemen - Rasulid Dynasty,terminusdb:///data/yetahir,terminusdb:///data/yerasul,terminusdb:///schema#present,Egypt - Mamluk Sultanate III,terminusdb:///data/egmambu,terminusdb:///data/a0d0100aeeb3f2908616545cdef...
942,system:unknown,system:unknown,Era of the War Lords,Banu Ziyad,terminusdb:///data/yewarld,terminusdb:///data/yeziyad,terminusdb:///schema#present,Ayyubid Sultanate,terminusdb:///data/egayyub,terminusdb:///data/b6a783d39265018c7a7c530f5ef...
943,system:unknown,system:unknown,Yemen - Neolithic,system:unknown,terminusdb:///data/yeyemnl,system:unknown,system:unknown,Minean Kingdom,terminusdb:///data/yemaink,system:unknown


In [13]:
view=WOQLView()
view.height(1000)
view.width(1200)
view.edges(["v:Predecessor", "v:Polity"], ["v:Polity", "v:Successor"], ["v:Polity", "v:Conflict"])
view.node("v:Predecessor").text("v:Plab").icon({"label": True, "color": [0,0,0]})
view.node("v:Successor").text("v:Slab").icon({"label": True, "color": [0,0,0]}).color([148,103,189])
view.node("v:Polity").text("v:Lab").icon({"label": True, "color": [0,0,0]}).size(20).color([188, 189, 34])
view.node("v:Polity", "v:Presence").of("terminusdb:///schema#present").color([23,190,207])
view.node("v:Polity", "v:Presence").of("terminusdb:///schema#absent").color([31,119,180])
view.node("v:Polity", "v:Presence").of("system:unknown").color([150,150,150])
view.node("v:Conflict").text("v:Blab").icon({"label": True, "color": [0,0,0]}).size(20).color([255, 0, 0])
view.show(result)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
view.export("viz",result)

In [20]:
conflict = "conflict"
seshat_result = (WQ().woql_and(
        WQ().triple("v:Conflict", "type", "scm:Battle"),
        WQ().triple("v:Conflict", "scm:belligerent", "v:Polity"),
        WQ().triple("v:Polity", "type", "scm:Polity"), 
        WQ().triple("v:Polity", "label", "v:Name")
        )
     ).execute(client)
df = WDF.result_to_df(seshat_result)
pandas.set_option('display.max_rows', df.shape[0]+1)
df

,Conflict,Name,Polity
0,terminusdb:///data/Battle_of_Callinicum,Sasanid Empire II,terminusdb:///data/irsasn2
1,terminusdb:///data/Battle_of_Callinicum,East Roman Empire,terminusdb:///data/trerom
2,terminusdb:///data/Battle_of_Vézeronce,Early Merovingian,terminusdb:///data/frmerve
3,terminusdb:///data/Battle_of_Amritsar_(1757),Durrani Empire,terminusdb:///data/afdurrn
4,terminusdb:///data/1548_capture_of_Aden,Ottoman Empire II,terminusdb:///data/trottm3
5,terminusdb:///data/Abbasid_Revolution,Umayyad Caliphate,terminusdb:///data/sycalum
6,terminusdb:///data/Action_of_30_September_1681,Spanish Empire,terminusdb:///data/eshabsb
7,terminusdb:///data/Adana_campaign,Egypt - Mamluk Sultanate III,terminusdb:///data/egmambu
8,terminusdb:///data/Adana_campaign,Ottoman Empire I,terminusdb:///data/trottm2
9,terminusdb:///data/Albanian-Turkish_Wars_(1432...,Ottoman Empire I,terminusdb:///data/trottm2


In [26]:
result = (WQ().woql_and(polity_query, date_query)
         ).execute(client)

df = WDF.result_to_df(result)
pandas.set_option('display.max_rows', df.shape[0]+1)
df

,Blab,Conflict,End_Integer,Lab,Plab,Polity,Predecessor,Presence,Slab,Start_Integer,Successor,Variable
0,system:unknown,system:unknown,561,Hepthalite Empire,Sassanid Empire I,terminusdb:///data/afhepht,terminusdb:///data/irsasn1,terminusdb:///schema#present,First Turk Khaganate,408,terminusdb:///data/mnturk1,terminusdb:///data/12cf48ebfc337d642c7ec0c17d5...
1,system:unknown,system:unknown,477,Kidarite Kingdom,Sassanid Empire I,terminusdb:///data/afkidar,terminusdb:///data/irsasn1,terminusdb:///schema#present,system:unknown,388,system:unknown,terminusdb:///data/2af16bd7b4fd7f12080a2d96681...
2,system:unknown,system:unknown,577,Northern Qi,system:unknown,terminusdb:///data/cnnqi,system:unknown,terminusdb:///schema#absent,system:unknown,550,system:unknown,terminusdb:///data/4968bfd3f014dc187f5f090524e...
3,Battle of Canhe Slope,terminusdb:///data/Battle_of_Canhe_Slope,534,Northern Wei,system:unknown,terminusdb:///data/cnnwei,system:unknown,terminusdb:///schema#absent,Western Wei,386,terminusdb:///data/cnwwei,terminusdb:///data/494b56226b93de758c7aaaef10a...
4,system:unknown,system:unknown,581,Northern Zhou,system:unknown,terminusdb:///data/cnnzhou,system:unknown,terminusdb:///schema#absent,system:unknown,557,system:unknown,terminusdb:///data/d79f266e5a4d35e3a249d1c74ab...
5,Battle of Huoyi,terminusdb:///data/Battle_of_Huoyi,618,Sui Dynasty,Northern Zhou,terminusdb:///data/cnsui,terminusdb:///data/cnnzhou,terminusdb:///schema#present,system:unknown,581,system:unknown,terminusdb:///data/584f4daa0c5bab69cc0a5972944...
6,Battle of Huoyi,terminusdb:///data/Battle_of_Huoyi,618,Sui Dynasty,Northern Zhou,terminusdb:///data/cnsui,terminusdb:///data/cnnzhou,terminusdb:///schema#absent,system:unknown,581,system:unknown,terminusdb:///data/c1ecec2c4c353a808095789b8d9...
7,system:unknown,system:unknown,577,Western Wei,system:unknown,terminusdb:///data/cnwwei,system:unknown,terminusdb:///schema#absent,system:unknown,535,system:unknown,terminusdb:///data/3db536e61694c55ecfeeb664c91...
8,system:unknown,system:unknown,1050,Neguanje,system:unknown,terminusdb:///data/conahua,system:unknown,terminusdb:///schema#absent,Tairona,250,terminusdb:///data/cotairo,terminusdb:///data/ec596da4d5031e3ced49eb296a7...
9,terminusdb:///data/Battle of Vézeronce,terminusdb:///data/Battle_of_Vézeronce,543,Early Merovingian,system:unknown,terminusdb:///data/frmerve,system:unknown,terminusdb:///schema#present,Middle Merovingian,481,terminusdb:///data/frmervm,terminusdb:///data/d1e75fa83004c98e5ca0ee3aab2...


In [27]:
view=WOQLView()
view.height(1000)
view.width(1200)
view.edges(["v:Predecessor", "v:Polity"], ["v:Polity", "v:Successor"], ["v:Polity", "v:Conflict"])
view.node("v:Predecessor").text("v:Plab").icon({"label": True, "color": [0,0,0]})
view.node("v:Successor").text("v:Slab").icon({"label": True, "color": [0,0,0]}).color([148,103,189])
view.node("v:Polity").text("v:Lab").icon({"label": True, "color": [0,0,0]}).size(20).color([188, 189, 34])
view.node("v:Polity", "v:Presence").of("terminusdb:///schema#present").color([23,190,207])
view.node("v:Polity", "v:Presence").of("terminusdb:///schema#absent").color([31,119,180])
view.node("v:Polity", "v:Presence").of("system:unknown").color([150,150,150])
view.node("v:Conflict").text("v:Blab").icon({"label": True, "color": [0,0,0]}).size(20).color([255, 0, 0])
view.show(result)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>